# Test 3: Creating reference files in the cloud, MUR 0.01 degree data

All work performed in the cloud, using earthaccess to locate file-objects and access endpoints.

Starts basic, creating a single reference file, then a combined reference file for a week. Next, creates a single reference files for a year using parallel computations (Coiled), then combines them into a single reference. No parallel methods used for the combining portion.

#### Results

* Successfully created single and combined reference files for a week. Takes about 3 seconds to create each single reference, and about a second for combined reference.
* Successfully parallelized the creation of 365 reference files on a distributed cluster, and combined them into a single reference. References were saved locally as JSONs. Creating the single references took 2.5 minutes using 50 VMs and cost $0.12. Creating the combined reference without parallel computing took 2 minutes. If this scales linearly, it would take 40 minutes to create the combined reference for the entire record.
* Combined year reference can be used to open the data and perform computations.

## Install packages

To install kerchunk, used
```
!pip install git+https://github.com/fsspec/kerchunk

```

In [38]:
import os
import fsspec
import kerchunk
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import ujson
import xarray as xr
import earthaccess
import coiled

## 1. Use earthaccess to get file-like objects and endpoints

In [26]:
earthaccess.login()
granule_info = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    temporal=("2019-01-01", "2019-12-31"),
    )

Granules found: 365


In [27]:
fobjs = earthaccess.open(granule_info)

Opening 365 granules, approx size: 187.42 GB
using endpoint: https://archive.podaac.earthdata.nasa.gov/s3credentials


QUEUEING TASKS | :   0%|          | 0/365 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/365 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/365 [00:00<?, ?it/s]

In [8]:
example_endpoint = fobjs[0].full_name
example_endpoint

's3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20190101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

## 2. Create references for a single MUR file, then a week of files

Starts off with a single file. Then create single refs for a week of files and combine them into a single reference.

### 2.1 Single MUR file

In [31]:
def single_ref_earthaccess(fobj):
    """
    Creates and returns a reference for a single file.
    
    Inputs
    ------
    fobj: earthaccess.store.EarthAccessFile object
        Obtained from a call to earthaccess.open(). Also has necessary information about file endpoint.
    """
    endpoint = fobj.full_name
    reference = SingleHdf5ToZarr(fobj, endpoint, inline_threshold=0).translate()
    return reference, endpoint # returns both the kerchunk reference and the path the file on podaac-ops-cumulus-protected

In [9]:
%%time
reference, endpoint = single_ref_earthaccess(fobjs[0])

CPU times: user 493 ms, sys: 133 ms, total: 626 ms
Wall time: 3.62 s


#### Open MUR file using the reference and perform basic computation

In [28]:
## Get AWS creds
fs = earthaccess.get_s3fs_session(daac="PODAAC")

In [13]:
%%time

## Open file using reference
data_ker = xr.open_dataset(
    "reference://", engine="zarr", 
    chunks={},
    backend_kwargs={
        "storage_options": 
            {
            "fo": reference,
             "remote_protocol": "s3",
             "remote_options": fs.storage_options
            },
         "consolidated": False
        }
    )
data_ker

CPU times: user 32.1 ms, sys: 3.53 ms, total: 35.6 ms
Wall time: 280 ms


<xarray.Dataset> Size: 23GB
Dimensions:           (time: 1, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 8B 2019-01-01T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] 5GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 3GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20190101T210000Z
    time_coverage_start:        20181231T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [14]:
%%time

## Basic comp:
data_ker['analysed_sst'].mean().compute()

CPU times: user 13.8 s, sys: 2 s, total: 15.8 s
Wall time: 10.8 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(286.95274384)

#### Open and process using netCDF backend for comparison

In [15]:
%%time
data_nc = xr.open_dataset(fobjs[0])

CPU times: user 88.3 ms, sys: 122 μs, total: 88.4 ms
Wall time: 299 ms


In [16]:
%%time
data_nc['analysed_sst'].mean().compute()

CPU times: user 10.1 s, sys: 4.49 s, total: 14.6 s
Wall time: 19.1 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(286.95274384)

### 2.2 Week of MUR files

Will create individual references for each day as well as a combined reference for the week.

In [18]:
%%time

## Single refs for each day
refs_week = []
for fo in fobjs[:7]:
    ref, ep = single_ref_earthaccess(fo)
    refs_week.append(ref)

CPU times: user 3.03 s, sys: 705 ms, total: 3.74 s
Wall time: 13.5 s


In [19]:
%%time

## Combined ref
mzz = MultiZarrToZarr(
    refs_week,
    remote_protocol="s3",
    remote_options=fs.storage_options,
    concat_dims=["time"]
)

ref_week_combined = mzz.translate()

CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 910 ms


In [21]:
%%time

## Open data with combined ref
data = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": ref_week_combined,
            "remote_protocol": "s3",
            "remote_options": fs.storage_options
            },
        "consolidated": False
        }
    )
data

CPU times: user 86.7 ms, sys: 5.55 ms, total: 92.2 ms
Wall time: 430 ms


<xarray.Dataset> Size: 163GB
Dimensions:           (time: 7, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 56B 2019-01-01T09:00:00 ... 2019-...
Data variables:
    analysed_sst      (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] 36GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 18GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20190101T210000Z
    time_coverage_start:        20181231T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

## 3. Create reference for a year of files using parallel computing

Parallelizes the function defined in section 2.1. Uses Coiled to perform work on a distributed cluster. Instead of keeping the references in memory, saves them as JSONs.

In [32]:
## Save reference JSONs in these directories:
dir_refs_indv = './reference_jsons_individual/'
dir_refs_comb = './reference_jsons_combined/'

In [ ]:
!mkdir $dir_refs_indv
!mkdir $dir_refs_comb

In [35]:
%%time

## --------------------------------------------
## Create single reference files with parallel computing using Coiled
## --------------------------------------------

# Wrap `create_single_ref` into coiled function:
single_ref_earthaccess_par = coiled.function(
    region="us-west-2", spot_policy="on-demand", 
    vm_type="t4g.large", n_workers=50
    )(single_ref_earthaccess)

# Begin computations:
fobjs_process = fobjs[:365]
results = single_ref_earthaccess_par.map(fobjs_process)

# Save results to JSONs as they become available:
for reference, endpoint in results:
    name_ref = dir_refs_indv + endpoint.split('/')[-1].replace('.nc', '.json')
    with open(name_ref, 'w') as outf:
        outf.write(ujson.dumps(reference))

single_ref_earthaccess_par.cluster.shutdown()

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│            ╷                                                                 │
│   Package  │ Note                                                            │
│ ╶──────────┼───────────────────────────────────────────────────────────────╴ │
│   kerchunk │ Wheel built from                                                │
│            │ /opt/coiled/env/.path-deps/kerchunk-0.2.5.post5-py3-none-any.   │
│            │ whl                                                             │
│            ╵                                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│             ╷                                                    ╷           │
│   Package   │ Error                                              │ Risk      │
│ ╶───────────┼────────────────────────────────────────────────────┼─────────╴ │
│   libcxxabi │ libcxxabi~=18.1.8 has no install candidate for     │ Warning   │
│             │ Python 3.12 linux-aarch64 on conda-forge           │           │
│             ╵                                                    ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

CPU times: user 4.58 s, sys: 198 ms, total: 4.78 s
Wall time: 2min 37s


In [36]:
## Get AWS creds
fs = earthaccess.get_s3fs_session(daac="PODAAC")

In [40]:
%%time

## --------------------------------------------
## Create combined reference file
## --------------------------------------------

ref_files_indv = [dir_refs_indv+f for f in os.listdir(dir_refs_indv) if f.endswith('.json')]
ref_files_indv.sort()
print(ref_files_indv[:5])

## Combined reference file
mzz = MultiZarrToZarr(
    ref_files_indv,
    remote_protocol="s3",
    remote_options=fs.storage_options,
    concat_dims=["time"], 
    )
ref_combined = mzz.translate()

 # Save reference info to JSON:
name_refcombined = dir_refs_comb + "MUR_1year_combined.json"
with open(name_refcombined, 'wb') as outf:
    outf.write(ujson.dumps(ref_combined).encode())

['./reference_jsons_individual/20190101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json', './reference_jsons_individual/20190102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json', './reference_jsons_individual/20190103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json', './reference_jsons_individual/20190104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json', './reference_jsons_individual/20190105090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json']
CPU times: user 6.71 s, sys: 669 ms, total: 7.37 s
Wall time: 1min 48s


#### Open data using reference file and perform some computations

In [41]:
%%time

## Open data with combined ref
data = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": name_refcombined,
            "remote_protocol": "s3",
            "remote_options": fs.storage_options
            },
        "consolidated": False
        }
    )
data

CPU times: user 1.31 s, sys: 115 ms, total: 1.42 s
Wall time: 1.65 s


<xarray.Dataset> Size: 10TB
Dimensions:           (time: 365, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 3kB 2019-01-01T09:00:00 ... 2019-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] 2TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 946GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sst_anomaly       (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20190101T210000Z
    time_coverage_start:        20181231T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [42]:
%%time
## Subset to single timestamp and small region and take mean:
data['analysed_sst'].sel(time=data["time"][-1], lat=slice(-10,10), lon=slice(-20,0)).mean().compute()

CPU times: user 224 ms, sys: 16.5 ms, total: 240 ms
Wall time: 488 ms


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(301.00789631)
Coordinates:
    time     datetime64[ns] 8B 2019-12-31T09:00:00

In [43]:
%%time
## Subset to 100 time stamps and region 2x the size, and take mean:
data['analysed_sst'].sel(time=data["time"][:100], lat=slice(-20,20), lon=slice(-20,10)).mean().compute()

CPU times: user 54.2 s, sys: 8.84 s, total: 1min 3s
Wall time: 1min 3s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(300.26490016)